a step by step demonstration for scripts/01.create_clean_scene.py

In [ ]:
import yaml
import numpy as np
import pickle

import scene_point_etk.argoverse2 as argoverse2
import scene_point_etk.scene_db as scene_db
import scene_point_etk.patch_db as patch_db
import scene_point_etk.visualization as visualization
import scene_point_etk.utils as scene_utils

import py_utils.utils as utils
import py_utils.voxel_grid as voxel_grid

In [ ]:
from importlib import reload

In [ ]:
with open("../scripts/configs/overall_configs.yaml", "r") as fd:
    configs = yaml.safe_load(fd)
list(configs.keys())

In [ ]:
log_id = argoverse2.list_log_ids()[0]
log_id

---
delete somethin on original scene

In [ ]:
scene = scene_db.OriginalScene(log_id)

In [ ]:
original_annots = argoverse2.Annotations(log_id)
dynamic_annots = scene.scene_details["annotations"]

static_annots = original_annots - dynamic_annots
static_annots

In [ ]:
camera_sequence = scene.camera_sequence
camera_sequence

In [ ]:
# filter out bounding boxes that are not visible in the camera sequence

bbox_vertices = static_annots.bounding_box_vertices()  # (N, 8, 3)
bbox_vertices = bbox_vertices.reshape(-1, 3)

point_masks = camera_sequence.FOV_mask(
    bbox_vertices,
    min_distance=0,
    max_distance=np.inf,
    verbose=True,
)
box_masks = point_masks.reshape(-1, 8)
box_masks = np.all(box_masks, axis=1)

filtered_annots = static_annots[box_masks]
filtered_annots

In [ ]:
delete_info = {
    "annotations": filtered_annots,
    "margin": configs["bbox_params"]["bbox_margin"],
    "indices": [],
}

_, mask = scene_db.get_deleted_pcd(
    scene.scene_pcd, delete_info, return_mask=True
)

In [ ]:
visualization.plot_multiple_pcds(
    (scene.pcd_xyz, scene.pcd_intensity),
    (scene.pcd_xyz[~mask], scene.pcd_intensity[~mask]),
)

---
add something on original scene

In [ ]:
patch_db.list_valid_patch_keys()[50:130:10]

In [ ]:
anchors = scene_utils.infer_anchor_points_by_av2_log_id(
    log_id,
    area_per_point=configs["anchor_params"]["area_per_point"],
    FOV_cameras=configs["anchor_params"]["FOV_cameras"],
    FOV_max_distance=configs["anchor_params"]["FOV_max_distance"],
    FOV_min_distance=configs["anchor_params"]["FOV_min_distance"],
)
anchors

In [ ]:
scene.camera_sequence.get_a_camera(0).transformation.shape

In [ ]:
# assign orientation to each anchor point
poses = scene.camera_sequence.get_a_camera(0).transformation  # (N, 4, 4)
poses_xyz = poses[:, :3, 3]  # (N, 3)
poses_euler = utils.R_to_euler(poses[:, :3, :3])  # (N, 3)

# get the closest position to assign the yaw angle
d = anchors[:, None, :2] - poses_xyz[:, :2]
d = np.linalg.norm(d, axis=-1)
I = np.argmin(d, axis=-1)
yaw_angles = poses_euler[I, 2]

# initialize anchors with yaw angles
anchors_euler = np.zeros((len(anchors), 3))
anchors_euler[:, 2] = yaw_angles

anchors_euler

In [ ]:
# assign patches to each anchor
N = len(anchors)
patch_keys = patch_db.list_valid_patch_keys()

selective_patches = np.random.randint(0, len(patch_keys), N)
selective_patches = [patch_keys[i] for i in selective_patches]
selective_patches

In [ ]:
# form the change_info/
add_info = {
    "patches": selective_patches,
    "anchor_xyzs": anchors,
    "anchor_eulers": anchors_euler,
    "z_offset": configs["anchor_params"]["ground_offset"],
    "voxel_size": configs["pcd_params"]["voxel_size"],
}

In [ ]:
add_pcd = scene_db.get_added_pcd(
    scene_pcd=scene.scene_pcd,
    add_info=add_info,
)

In [ ]:
merged_pcd = np.concatenate([scene.scene_pcd, add_pcd])
merged_xyz = np.vstack([merged_pcd["x"], merged_pcd["y"], merged_pcd["z"]]).T

In [ ]:
visualization.plot_multiple_pcds(
    (scene.pcd_xyz, scene.pcd_intensity),
    (merged_xyz, merged_pcd["intensity"]),
)

---
combine both delete and add

In [ ]:
change_info = {
    "add": add_info,
    "delete": delete_info,
    "voxel_grid": configs["pcd_params"]["voxel_size"],
}

new_scene_pcd = scene_db.apply_change_info_to_target_pcd(
    scene.scene_pcd, change_info
)

In [ ]:
new_scene = scene_db.EditedScene(log_id, "v0")
new_scene.scene_pcd = new_scene_pcd
new_scene.scene_details = change_info

In [ ]:
new_scene.camera_sequence

In [ ]:
for camera in new_scene.cameras:
    new_scene.process_camera(camera)

In [ ]:
scene_db.set_scene_root()

In [ ]:
new_scene = scene_db.EditedScene(log_id, "v0")

In [ ]:
visualization.plot_multiple_pcds(
    (scene.pcd_xyz, scene.pcd_intensity),
    (new_scene.pcd_xyz, new_scene.pcd_intensity),
)

In [ ]:
scene_db.SCENE_MAP